# Glass Sponge (Hexactinellid) Structural Motifs — Visualizer

This notebook is a small *interactive app* (built with **Gradio + Plotly**) that lets you visualize
parametric, schematic geometries inspired by classic structural biology / mechanics papers on **glass sea sponges** (Hexactinellida), especially *Euplectella aspergillum*:

- **Macro (whole skeleton):** cylindrical **square-grid lattice** with an **alternating (checkerboard) diagonal bracing** pattern, plus **external spiral ridges**, a **terminal sieve plate**, and a **basal anchor tuft**.
- **Micro/Meso (components):** laminated **spicule cross-sections** (alternating silica/organic layers around an axial filament), and **composite struts** (bundled spicules embedded in a silica matrix).
- **Element level:** a simplified **non‑planar cruciform spicule** element.

> These are **schematic** models meant for intuition and exploration (not anatomical reconstructions).

---

## Files created by this project

- `glass_sponge_algorithms.py` — the reusable geometry + plotting algorithms
- This notebook — a UI front-end around those algorithms

---

## Quick start

Run all cells, then use the tabs in the UI.


In [1]:
# Imports and setup
import numpy as np
import plotly.graph_objects as go
import gradio as gr

from glass_sponge_algorithms import (
    # parameter containers
    LatticeParams, RidgeParams, SievePlateParams, AnchorBundleParams,
    SpiculeLamellaParams, CompositeBeamParams, CruciformSpiculeParams,
    NanoparticleRingParams, OpticalSpiculeParams, BasaliaSpiculeParams,
    IndentationFractureParams,

    # figure builders + metrics
    full_skeleton_figure, lattice_figure,
    spicule_cross_section_figure, composite_beam_cross_section_figure,
    cruciform_spicule_figure, anchor_bundle_figure,
    nanoparticle_rings_figure,
    optical_spicule_index_profile_figure, optical_spicule_index_map_figure, optical_spicule_metrics,
    basalia_spicule_figure, normalized_cracking_load,
)

print("Imports OK.")


Imports OK.


In [2]:
# Render helpers (used by the Gradio UI below)
#
# NOTE: This notebook intentionally uses simplified parametric geometry.
# The goal is to help explore architectural motifs (macro→micro→nano) reported in the papers.


def _maybe_none_if_equal(a: float, b: float, tol: float = 1e-9):
    return None if abs(float(a) - float(b)) < tol else float(b)


def render_macro_skeleton(
    radius_base: float,
    taper_factor: float,
    height: float,
    n_theta: int,
    n_z: int,
    diagonal_mode: str,
    paired_diagonals: bool,
    interpenetrating: bool,
    # ridges
    show_ridges: bool,
    n_ridges: int,
    ridge_pitch_base: float,
    ridge_pitch_top: float,
    ridge_height_base: float,
    ridge_height_top: float,
    ridge_start_frac: float,
    ridge_handedness: str,
    # sieve plate
    show_sieve: bool,
    sieve_rings: int,
    sieve_spokes: int,
    sieve_dome: float,
    # anchor tuft
    anchor_fibers: int,
    anchor_length: float,
    anchor_spread: float,
    anchor_waviness: float,
    seed: int,
    show_axes: bool,
):
    radius_top = None
    if abs(float(taper_factor) - 1.0) > 1e-6:
        radius_top = float(radius_base) * float(taper_factor)

    lattice = LatticeParams(
        radius=float(radius_base),
        radius_top=radius_top,
        height=float(height),
        n_theta=int(n_theta),
        n_z=int(n_z),
        diagonal_mode=diagonal_mode,
        diagonal_pair_offset=0.02 if paired_diagonals else 0.0,
        interpenetrating=bool(interpenetrating),
        secondary_theta_offset=0.5,
        secondary_radial_offset=0.0,
    )

    ridges = None
    if show_ridges:
        ridges = RidgeParams(
            n_ridges=int(n_ridges),
            pitch=float(ridge_pitch_base),
            pitch_top=_maybe_none_if_equal(ridge_pitch_base, ridge_pitch_top),
            ridge_height=float(ridge_height_base),
            ridge_height_top=_maybe_none_if_equal(ridge_height_base, ridge_height_top),
            start_z_frac=float(ridge_start_frac),
            handedness=ridge_handedness,
        )

    sieve = None
    if show_sieve:
        sieve = SievePlateParams(
            n_rings=int(sieve_rings),
            n_spokes=int(sieve_spokes),
            dome_height=float(sieve_dome),
        )

    anchor = AnchorBundleParams(
        n_fibers=int(anchor_fibers),
        length=float(anchor_length),
        spread=float(anchor_spread),
        waviness=float(anchor_waviness),
        seed=int(seed),
    )

    fig = full_skeleton_figure(
        lattice=lattice,
        ridges=ridges,
        sieve=sieve,
        anchor=anchor,
        show_axes=bool(show_axes),
    )
    return fig


def render_lattice_only(
    radius_base: float,
    taper_factor: float,
    height: float,
    n_theta: int,
    n_z: int,
    diagonal_mode: str,
    paired_diagonals: bool,
    interpenetrating: bool,
    # ridges
    show_ridges: bool,
    n_ridges: int,
    ridge_pitch_base: float,
    ridge_pitch_top: float,
    ridge_height_base: float,
    ridge_height_top: float,
    ridge_start_frac: float,
    ridge_handedness: str,
    # sieve plate
    show_sieve: bool,
    sieve_rings: int,
    sieve_spokes: int,
    sieve_dome: float,
    show_axes: bool,
):
    radius_top = None
    if abs(float(taper_factor) - 1.0) > 1e-6:
        radius_top = float(radius_base) * float(taper_factor)

    lattice = LatticeParams(
        radius=float(radius_base),
        radius_top=radius_top,
        height=float(height),
        n_theta=int(n_theta),
        n_z=int(n_z),
        diagonal_mode=diagonal_mode,
        diagonal_pair_offset=0.02 if paired_diagonals else 0.0,
        interpenetrating=bool(interpenetrating),
        secondary_theta_offset=0.5,
        secondary_radial_offset=0.0,
    )

    ridges = None
    if show_ridges:
        ridges = RidgeParams(
            n_ridges=int(n_ridges),
            pitch=float(ridge_pitch_base),
            pitch_top=_maybe_none_if_equal(ridge_pitch_base, ridge_pitch_top),
            ridge_height=float(ridge_height_base),
            ridge_height_top=_maybe_none_if_equal(ridge_height_base, ridge_height_top),
            start_z_frac=float(ridge_start_frac),
            handedness=ridge_handedness,
        )

    sieve = None
    if show_sieve:
        sieve = SievePlateParams(
            n_rings=int(sieve_rings),
            n_spokes=int(sieve_spokes),
            dome_height=float(sieve_dome),
        )

    fig = lattice_figure(lattice=lattice, ridges=ridges, sieve=sieve, show_axes=bool(show_axes))
    return fig


def render_spicule_cross_section(
    outer_radius: float,
    n_layers: int,
    organic_thickness: float,
    silica_thickness_inner: float,
    silica_thickness_outer: float,
    axial_filament_radius: float,
    thickness_profile: str,
    powerlaw_exp: float,
):
    p = SpiculeLamellaParams(
        outer_radius=float(outer_radius),
        n_silica_layers=int(n_layers),
        organic_thickness=float(organic_thickness),
        silica_thickness_inner=float(silica_thickness_inner),
        silica_thickness_outer=float(silica_thickness_outer),
        axial_filament_radius=float(axial_filament_radius),
        thickness_profile=thickness_profile,
        powerlaw_exp=float(powerlaw_exp),
    )
    return spicule_cross_section_figure(p)


def render_composite_beam(
    beam_radius: float,
    n_spicules: int,
    spicule_radius_mean: float,
    spicule_radius_std: float,
    padding: float,
    seed: int,
):
    p = CompositeBeamParams(
        beam_radius=float(beam_radius),
        n_spicules=int(n_spicules),
        spicule_radius_mean=float(spicule_radius_mean),
        spicule_radius_std=float(spicule_radius_std),
        packing_padding=float(padding),
        seed=int(seed),
    )
    return composite_beam_cross_section_figure(p)


def render_anchor_bundle(
    n_fibers: int,
    length: float,
    spread: float,
    waviness: float,
    seed: int,
):
    p = AnchorBundleParams(
        n_fibers=int(n_fibers),
        length=float(length),
        spread=float(spread),
        waviness=float(waviness),
        seed=int(seed),
    )
    return anchor_bundle_figure(p)


def render_cruciform(
    ray_length: float,
    vertical_ratio: float,
    tilt_degrees: float,
    points_per_ray: int,
):
    p = CruciformSpiculeParams(
        ray_length=float(ray_length),
        vertical_to_horizontal_ratio=float(vertical_ratio),
        tilt_degrees=float(tilt_degrees),
        n_points_per_ray=int(points_per_ray),
    )
    return cruciform_spicule_figure(p)


def render_nanoparticle_rings(
    outer_radius: float,
    core_radius: float,
    n_rings: int,
    particles_per_ring: int,
    radial_jitter: float,
    angular_jitter: float,
    scale_marker: bool,
    marker_size_inner: float,
    marker_size_outer: float,
    subparticles_per_particle: int,
    subparticle_cloud_radius: float,
    seed: int,
):
    p = NanoparticleRingParams(
        outer_radius=float(outer_radius),
        axial_filament_radius=float(core_radius),
        n_rings=int(n_rings),
        particles_per_ring=int(particles_per_ring),
        radial_jitter=float(radial_jitter),
        angular_jitter=float(angular_jitter),
        seed=int(seed),
        scale_marker_by_radius=bool(scale_marker),
        marker_size_inner=float(marker_size_inner),
        marker_size_outer=float(marker_size_outer),
        subparticles_per_particle=int(subparticles_per_particle),
        subparticle_cloud_radius=float(subparticle_cloud_radius),
    )
    return nanoparticle_rings_figure(p)


def render_optical_spicule(
    outer_radius_um: float,
    core_diameter_um: float,
    central_cylinder_diameter_um: float,
    n_core_min: float,
    n_core_max: float,
    n_central_cylinder: float,
    n_shell_inner: float,
    n_shell_outer: float,
    shell_osc_amp: float,
    shell_layer_spacing_um: float,
    wavelength_um: float,
    map_grid: int,
):
    p = OpticalSpiculeParams(
        outer_radius_um=float(outer_radius_um),
        axial_filament_radius_um=0.25,
        core_radius_um=float(core_diameter_um) / 2.0,
        central_cylinder_radius_um=float(central_cylinder_diameter_um) / 2.0,
        n_core_min=float(n_core_min),
        n_core_max=float(n_core_max),
        n_central_cylinder=float(n_central_cylinder),
        n_shell_inner=float(n_shell_inner),
        n_shell_outer=float(n_shell_outer),
        shell_osc_amp=float(shell_osc_amp),
        shell_layer_spacing_um=float(shell_layer_spacing_um),
    )

    prof = optical_spicule_index_profile_figure(p)
    mp = optical_spicule_index_map_figure(p, n_grid=int(map_grid))

    metrics = optical_spicule_metrics(p, wavelength_um=float(wavelength_um))
    md = (
        f"**Approx. numerical aperture (NA):** {metrics['NA']:.4f}\n\n"
        f"**Acceptance half-angle in air:** {metrics['theta_air_deg']:.1f}°\n\n"
        f"**V-number** (core radius a={p.core_radius_um:.2f} µm, λ={wavelength_um:.3f} µm): {metrics['V']:.1f}"
    )
    return prof, mp, md


def render_basalia_spicule(
    length: float,
    radius: float,
    barbed_fraction: float,
    n_barbs: int,
    barb_length: float,
    tip_spines: int,
    tip_spine_length: float,
    waviness: float,
    seed: int,
    show_axes: bool,
):
    p = BasaliaSpiculeParams(
        length=float(length),
        radius=float(radius),
        barbed_fraction=float(barbed_fraction),
        n_barbs=int(n_barbs),
        barb_length=float(barb_length),
        tip_spines=int(tip_spines),
        tip_spine_length=float(tip_spine_length),
        waviness=float(waviness),
        seed=int(seed),
    )
    return basalia_spicule_figure(p, show_axes=bool(show_axes))


def render_indentation_compare(
    # case A (e.g., monolithic/core)
    Kc_A: float,
    E_A: float,
    H_A: float,
    # case B (e.g., laminated region)
    Kc_B: float,
    E_B: float,
    H_B: float,
):
    a = IndentationFractureParams(Kc_MPa_sqrtm=float(Kc_A), E_GPa=float(E_A), H_GPa=float(H_A))
    b = IndentationFractureParams(Kc_MPa_sqrtm=float(Kc_B), E_GPa=float(E_B), H_GPa=float(H_B))
    Pc_A = normalized_cracking_load(a)
    Pc_B = normalized_cracking_load(b)

    ratio = Pc_B / Pc_A if Pc_A > 0 else float("nan")

    md = (
        f"### Normalized cracking-load scaling (schematic)\n\n"
        f"Using **Pc ∝ Kc⁴ / (E²·H)** (units cancel for *relative* comparisons):\n\n"
        f"- Case A: Pc ~ {Pc_A:.3e}\n"
        f"- Case B: Pc ~ {Pc_B:.3e}\n\n"
        f"**Ratio (B / A): {ratio:.2f}×**\n\n"
        f"Interpretation: increasing fracture toughness Kc has a strong (4th-power) effect on this scaling."
    )
    return md


In [ ]:
# Interactive UI (Gradio)

with gr.Blocks() as demo:
    gr.Markdown(
        """
# Glass Sponge Structures Visualizer (papers-integrated)

This UI provides **schematic, parametric** visualizations of hierarchical motifs described in the
uploaded glass-sponge papers (macro lattice + ridges, laminated spicules, nanoparticle organization,
and optical-waveguide spicules).

**What changed vs the previous notebook**
- Macro lattice can be **tapered** (base→apex radius).
- Optional **interpenetrating lattice** (two-grid proxy).
- Ridges now support **start height**, **pitch gradient**, and **height gradient**.
- Added an **optical spicule** tab (refractive-index profile + map + NA/V-number).
- Added a **basalia barbs** tab.
- Added a simple **indentation/fracture scaling** tab for comparing regions.
"""
    )

    with gr.Tab("Macro skeleton"):
        gr.Markdown("Cylindrical lattice + optional ridges + optional sieve plate + anchor tuft.")
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### Lattice")
                radius = gr.Slider(0.5, 2.0, value=1.0, step=0.05, label="Base radius (arb. units)")
                taper = gr.Slider(1.0, 2.5, value=1.7, step=0.05, label="Taper factor (top_radius / base_radius)")
                height = gr.Slider(2.0, 10.0, value=5.0, step=0.1, label="Height")
                nth = gr.Slider(8, 80, value=36, step=1, label="Vertical struts (around circumference)")
                nz = gr.Slider(6, 140, value=70, step=1, label="Horizontal rings (along height)")
                diag_mode = gr.Dropdown(["none", "checkerboard", "all_cells"], value="checkerboard", label="Diagonal bracing mode")
                paired = gr.Checkbox(value=True, label="Paired diagonals (visual offset)")
                inter = gr.Checkbox(value=True, label="Interpenetrating (offset) second lattice")
                show_axes = gr.Checkbox(value=False, label="Show axes")
            with gr.Column(scale=1):
                gr.Markdown("### External ridges")
                show_ridges = gr.Checkbox(value=True, label="Show ridges")
                nr = gr.Slider(1, 12, value=4, step=1, label="Number of ridge families")
                pitch0 = gr.Slider(0.5, 12.0, value=6.0, step=0.1, label="Pitch (base) — z per revolution")
                pitch1 = gr.Slider(0.5, 12.0, value=4.0, step=0.1, label="Pitch (top) — smaller = tighter helix")
                rh0 = gr.Slider(0.0, 0.25, value=0.03, step=0.005, label="Ridge height (base)")
                rh1 = gr.Slider(0.0, 0.35, value=0.16, step=0.005, label="Ridge height (top)")
                rstart = gr.Slider(0.0, 0.5, value=0.15, step=0.01, label="Ridge start height (fraction of body)")
                handed = gr.Dropdown(["both", "left", "right"], value="both", label="Ridge handedness")
                gr.Markdown("### Terminal sieve plate")
                show_sieve = gr.Checkbox(value=True, label="Show sieve plate")
                sr = gr.Slider(1, 10, value=6, step=1, label="Concentric rings")
                ss = gr.Slider(6, 120, value=24, step=1, label="Spokes")
                dome = gr.Slider(0.0, 0.6, value=0.25, step=0.02, label="Dome height (convexity)")
                gr.Markdown("### Anchor tuft")
                af = gr.Slider(10, 250, value=80, step=1, label="Fibers")
                al = gr.Slider(0.5, 8.0, value=2.5, step=0.1, label="Length")
                asp = gr.Slider(0.1, 4.0, value=1.0, step=0.05, label="Spread")
                aw = gr.Slider(0.0, 1.0, value=0.25, step=0.05, label="Waviness")
                seed = gr.Slider(0, 50, value=0, step=1, label="Seed")
                btn = gr.Button("Render macro skeleton")
            with gr.Column(scale=2):
                out1 = gr.Plot(label="3D view")

        btn.click(
            render_macro_skeleton,
            inputs=[radius, taper, height, nth, nz, diag_mode, paired, inter,
                    show_ridges, nr, pitch0, pitch1, rh0, rh1, rstart, handed,
                    show_sieve, sr, ss, dome,
                    af, al, asp, aw, seed, show_axes],
            outputs=out1,
        )

    with gr.Tab("Lattice only"):
        gr.Markdown("Lattice with optional ridges + sieve plate (no anchor tuft).")
        with gr.Row():
            with gr.Column(scale=1):
                radius2 = gr.Slider(0.5, 2.0, value=1.0, step=0.05, label="Base radius")
                taper2 = gr.Slider(1.0, 2.5, value=1.7, step=0.05, label="Taper factor")
                height2 = gr.Slider(2.0, 10.0, value=5.0, step=0.1, label="Height")
                nth2 = gr.Slider(8, 80, value=36, step=1, label="Vertical struts")
                nz2 = gr.Slider(6, 140, value=70, step=1, label="Horizontal rings")
                diag_mode2 = gr.Dropdown(["none", "checkerboard", "all_cells"], value="checkerboard", label="Diagonal mode")
                paired2 = gr.Checkbox(value=True, label="Paired diagonals")
                inter2 = gr.Checkbox(value=True, label="Interpenetrating second lattice")
                show_axes2 = gr.Checkbox(value=False, label="Show axes")
            with gr.Column(scale=1):
                show_ridges2 = gr.Checkbox(value=True, label="Show ridges")
                nr2 = gr.Slider(1, 12, value=4, step=1, label="Ridge families")
                pitch02 = gr.Slider(0.5, 12.0, value=6.0, step=0.1, label="Pitch base")
                pitch12 = gr.Slider(0.5, 12.0, value=4.0, step=0.1, label="Pitch top")
                rh02 = gr.Slider(0.0, 0.25, value=0.03, step=0.005, label="Height base")
                rh12 = gr.Slider(0.0, 0.35, value=0.16, step=0.005, label="Height top")
                rstart2 = gr.Slider(0.0, 0.5, value=0.15, step=0.01, label="Start frac")
                handed2 = gr.Dropdown(["both", "left", "right"], value="both", label="Handedness")
                show_sieve2 = gr.Checkbox(value=True, label="Show sieve plate")
                sr2 = gr.Slider(1, 10, value=6, step=1, label="Rings")
                ss2 = gr.Slider(6, 120, value=24, step=1, label="Spokes")
                dome2 = gr.Slider(0.0, 0.6, value=0.25, step=0.02, label="Dome height")
                btn2 = gr.Button("Render lattice")
            with gr.Column(scale=2):
                out2 = gr.Plot(label="3D view")

        btn2.click(
            render_lattice_only,
            inputs=[radius2, taper2, height2, nth2, nz2, diag_mode2, paired2, inter2,
                    show_ridges2, nr2, pitch02, pitch12, rh02, rh12, rstart2, handed2,
                    show_sieve2, sr2, ss2, dome2, show_axes2],
            outputs=out2,
        )

    with gr.Tab("Spicule cross-section (lamellae)"):
        gr.Markdown("Concentric silica lamellae separated by thin organic interlayers (schematic).")
        with gr.Row():
            with gr.Column(scale=1):
                orad = gr.Slider(0.3, 2.0, value=1.0, step=0.05, label="Outer radius")
                nlay = gr.Slider(1, 60, value=18, step=1, label="Silica layers")
                orgt = gr.Slider(0.0, 0.05, value=0.003, step=0.0005, label="Organic interlayer thickness")
                tin = gr.Slider(0.01, 0.4, value=0.14, step=0.005, label="Silica thickness (inner)")
                tout = gr.Slider(0.005, 0.2, value=0.02, step=0.002, label="Silica thickness (outer)")
                afr = gr.Slider(0.0, 0.2, value=0.05, step=0.005, label="Axial filament radius")
                prof = gr.Dropdown(["linear", "powerlaw"], value="linear", label="Thickness profile")
                pexp = gr.Slider(0.5, 3.0, value=1.4, step=0.05, label="Power-law exponent (if used)")
                btn3 = gr.Button("Render")
            with gr.Column(scale=2):
                out3 = gr.Plot(label="2D cross-section")
        btn3.click(render_spicule_cross_section, inputs=[orad, nlay, orgt, tin, tout, afr, prof, pexp], outputs=out3)

    with gr.Tab("Optical waveguide spicule"):
        gr.Markdown("Basalia spicule refractive-index profile/map (schematic) + derived NA/V-number.")
        with gr.Row():
            with gr.Column(scale=1):
                or_um = gr.Slider(20.0, 120.0, value=50.0, step=1.0, label="Outer radius (µm)")
                core_d = gr.Slider(0.5, 3.0, value=1.5, step=0.05, label="High-index core diameter (µm)")
                cc_d = gr.Slider(10.0, 30.0, value=20.0, step=0.5, label="Central-cylinder diameter (µm)")
                n1min = gr.Slider(1.43, 1.48, value=1.45, step=0.001, label="n_core (min)")
                n1max = gr.Slider(1.44, 1.50, value=1.48, step=0.001, label="n_core (max)")
                ncc = gr.Slider(1.40, 1.44, value=1.425, step=0.001, label="n_central_cylinder")
                ns_in = gr.Slider(1.42, 1.44, value=1.433, step=0.001, label="n_shell (inner)")
                ns_out = gr.Slider(1.43, 1.45, value=1.438, step=0.001, label="n_shell (outer)")
                osc = gr.Slider(0.0, 0.002, value=0.0008, step=0.0001, label="Shell oscillation amplitude")
                spacing = gr.Slider(0.5, 1.5, value=0.9, step=0.05, label="Shell layer spacing (µm)")
                lam = gr.Slider(0.4, 1.0, value=0.633, step=0.005, label="Wavelength λ (µm) for V-number")
                grid = gr.Slider(80, 360, value=220, step=10, label="Map resolution (grid)")
                btnO = gr.Button("Render optical profile")
            with gr.Column(scale=2):
                outO1 = gr.Plot(label="Index profile n(r)")
                outO2 = gr.Plot(label="Index map (cross-section)")
                outO3 = gr.Markdown()

        btnO.click(
            render_optical_spicule,
            inputs=[or_um, core_d, cc_d, n1min, n1max, ncc, ns_in, ns_out, osc, spacing, lam, grid],
            outputs=[outO1, outO2, outO3],
        )

    with gr.Tab("Nanoparticle rings"):
        gr.Markdown("2D schematic of silica nanoparticle rings; optionally scale particle size with radius and add subparticle clouds.")
        with gr.Row():
            with gr.Column(scale=1):
                nor = gr.Slider(0.3, 2.0, value=1.0, step=0.05, label="Outer radius")
                ncore = gr.Slider(0.0, 0.3, value=0.08, step=0.01, label="Core (axial filament) radius")
                nring = gr.Slider(2, 30, value=10, step=1, label="Rings")
                ppr = gr.Slider(6, 120, value=40, step=1, label="Particles per ring")
                rjit = gr.Slider(0.0, 0.08, value=0.02, step=0.005, label="Radial jitter")
                ajit = gr.Slider(0.0, 0.2, value=0.04, step=0.01, label="Angular jitter")
                scale_m = gr.Checkbox(value=True, label="Scale marker size with radius")
                ms_in = gr.Slider(2, 20, value=6, step=1, label="Marker size (inner)")
                ms_out = gr.Slider(2, 30, value=16, step=1, label="Marker size (outer)")
                subn = gr.Slider(0, 30, value=0, step=1, label="Subparticles per particle (0 = off)")
                subr = gr.Slider(0.0, 0.05, value=0.01, step=0.002, label="Subparticle cloud radius")
                nseed = gr.Slider(0, 50, value=0, step=1, label="Seed")
                btnN = gr.Button("Render")
            with gr.Column(scale=2):
                outN = gr.Plot(label="2D view")
        btnN.click(render_nanoparticle_rings, inputs=[nor, ncore, nring, ppr, rjit, ajit, scale_m, ms_in, ms_out, subn, subr, nseed], outputs=outN)

    with gr.Tab("Composite strut"):
        gr.Markdown("Bundle of spicules embedded in a matrix (cross-section schematic).")
        with gr.Row():
            with gr.Column(scale=1):
                br = gr.Slider(0.3, 2.0, value=1.0, step=0.05, label="Beam radius")
                ns = gr.Slider(1, 80, value=20, step=1, label="Spicules in bundle")
                rm = gr.Slider(0.02, 0.4, value=0.12, step=0.01, label="Mean spicule radius")
                rs = gr.Slider(0.0, 0.2, value=0.03, step=0.005, label="Spicule radius std")
                pad = gr.Slider(0.0, 0.05, value=0.01, step=0.005, label="Padding")
                seedC = gr.Slider(0, 50, value=0, step=1, label="Random seed")
                btn4 = gr.Button("Render")
            with gr.Column(scale=2):
                out4 = gr.Plot(label="2D cross-section")
        btn4.click(render_composite_beam, inputs=[br, ns, rm, rs, pad, seedC], outputs=out4)

    with gr.Tab("Anchor tuft"):
        gr.Markdown("Flexible cluster of anchor spicules (schematic).")
        with gr.Row():
            with gr.Column(scale=1):
                nf = gr.Slider(10, 250, value=80, step=1, label="Fibers")
                al2 = gr.Slider(0.5, 8.0, value=2.5, step=0.1, label="Length")
                asp2 = gr.Slider(0.1, 4.0, value=1.0, step=0.05, label="Spread")
                aw2 = gr.Slider(0.0, 1.0, value=0.25, step=0.05, label="Waviness")
                sd = gr.Slider(0, 50, value=0, step=1, label="Seed")
                btn5 = gr.Button("Render")
            with gr.Column(scale=2):
                out5 = gr.Plot(label="3D view")
        btn5.click(render_anchor_bundle, inputs=[nf, al2, asp2, aw2, sd], outputs=out5)

    with gr.Tab("Cruciform spicule element"):
        gr.Markdown("Non-planar cruciform element (schematic building block).")
        with gr.Row():
            with gr.Column(scale=1):
                rl = gr.Slider(0.2, 3.0, value=1.0, step=0.1, label="Horizontal ray length")
                vr = gr.Slider(1.0, 4.0, value=2.0, step=0.1, label="Vertical/horizontal length ratio")
                tilt = gr.Slider(0.0, 45.0, value=20.0, step=1.0, label="Non-planarity (tilt degrees)")
                npr = gr.Slider(5, 120, value=30, step=1, label="Points per ray")
                btn6 = gr.Button("Render")
            with gr.Column(scale=2):
                out6 = gr.Plot(label="3D view")
        btn6.click(render_cruciform, inputs=[rl, vr, tilt, npr], outputs=out6)

    with gr.Tab("Basalia spicule barbs"):
        gr.Markdown("Schematic basalia spicule: smooth distal region + barbed proximal region + terminal tip spines.")
        with gr.Row():
            with gr.Column(scale=1):
                bl = gr.Slider(1.0, 8.0, value=3.0, step=0.1, label="Length")
                brad = gr.Slider(0.01, 0.2, value=0.05, step=0.005, label="Radius")
                bf = gr.Slider(0.0, 0.8, value=0.35, step=0.01, label="Barbed fraction (near base)")
                nb = gr.Slider(0, 300, value=80, step=1, label="Number of barbs")
                bL = gr.Slider(0.0, 0.3, value=0.08, step=0.005, label="Barb length")
                ts = gr.Slider(0, 20, value=6, step=1, label="Tip spines")
                tL = gr.Slider(0.0, 0.5, value=0.15, step=0.01, label="Tip spine length")
                wav = gr.Slider(0.0, 1.0, value=0.0, step=0.05, label="Waviness")
                sdb = gr.Slider(0, 50, value=0, step=1, label="Seed")
                axesB = gr.Checkbox(value=False, label="Show axes")
                btnB = gr.Button("Render basalia spicule")
            with gr.Column(scale=2):
                outB = gr.Plot(label="3D view")
        btnB.click(render_basalia_spicule, inputs=[bl, brad, bf, nb, bL, ts, tL, wav, sdb, axesB], outputs=outB)

    with gr.Tab("Indentation / fracture scaling"):
        gr.Markdown("Compare two regions using normalized Pc ∝ Kc⁴/(E²·H).")
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("**Case A** (e.g., monolithic / core)")
                KcA = gr.Slider(0.1, 1.5, value=0.35, step=0.01, label="Kc (MPa·√m)")
                EA = gr.Slider(5.0, 80.0, value=35.0, step=0.5, label="E (GPa)")
                HA = gr.Slider(0.5, 20.0, value=5.0, step=0.1, label="H (GPa)")
                gr.Markdown("**Case B** (e.g., laminated region)")
                KcB = gr.Slider(0.1, 2.0, value=0.84, step=0.01, label="Kc (MPa·√m)")
                EB = gr.Slider(5.0, 80.0, value=30.0, step=0.5, label="E (GPa)")
                HB = gr.Slider(0.5, 20.0, value=4.5, step=0.1, label="H (GPa)")
                btnI = gr.Button("Compute")
            with gr.Column(scale=2):
                outI = gr.Markdown()
        btnI.click(render_indentation_compare, inputs=[KcA, EA, HA, KcB, EB, HB], outputs=outI)

    gr.Markdown(
        "### Notes\n"
        "- The geometry is **parametric and schematic**. The goal is to explore architectural ideas described in the papers.\n"
        "- If you want CAD meshes / solids, the functions in `glass_sponge_algorithms.py` are the right place to extend."
    )

# Launch the UI.
# In notebooks, some Gradio versions don't auto-render a Blocks object unless you call .launch().
try:
    demo.launch(inline=True, debug=True, share=False, prevent_thread_lock=True)
except TypeError:
    # Fallback for older/newer Gradio versions with different launch() signatures
    try:
        demo.launch(debug=True, share=False, prevent_thread_lock=True)
    except TypeError:
        demo.launch(debug=True, share=False)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
